**кластеризатор текстов**

SnowballStemmer + TFIDF + DBSCAN

_Евгений Борисов <esborisov@sevsu.ru>_

In [1]:
import pandas as pd
from tqdm.auto import tqdm

pd.options.display.precision = 2 
pd.options.display.max_colwidth = 200 
tqdm.pandas()

## загрузка данных

датасет news.pkl.gz   
https://disk.yandex.ru/d/8_T_XITkZ4gKAw

In [2]:
# загружаем тексты
data = pd.read_pickle('news.pkl.gz')
display( len(data) )
display(  data.sample(2) )

3196

,text,tag
1335,"В калифорнийском метро нашли останки мастодонтов возрастом 10 тысяч лет\n\n2 декабря 2016 в 18:37\n\nLenta.ru\n\nВо время работ по расширению метро Лос-Анджелеса, штат Калифорния, рабочие наткнули...",tech
321,"Фестиваль военно-патриотических фильмов проходит в Подмосковье\n(3)""Волоколамский рубеж"" - так называется международный фестиваль\nвоенно-патриотического фильма им. Сергея Бондарчука, который втор...",culture


In [3]:
display( len( data.drop_duplicates('text') ) )

3196

## токенайзер

In [4]:
# NLTK package manager
# import nltk
# nltk.download()

In [5]:
# import re
# from nltk.tokenize import word_tokenize as nltk_tokenize_word

# def tokenizer(text):
#     return [
#             t for t in nltk_tokenize_word( # разбиваем текст на слова
#                 re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
#                 language='russian'
#             ) 
#         ]

In [6]:
# import re
# from nltk.tokenize import word_tokenize as nltk_tokenize_word
# from nltk.corpus import stopwords as nltk_stopwords

# stopwords = set(nltk_stopwords.words('russian'))

# def tokenizer(text,stopwords=stopwords):
#     return [
#             t for t in nltk_tokenize_word( # разбиваем текст на слова
#                 re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
#                 language='russian'
#             ) 
#             if not (
#                False
#                or (len(t)<3) # выкидываем очень короткие слова
#                or re.match(r'^[^a-zA-ZЁёА-я]+$', t) # выкидываем токены не содержащие букв
#                or re.match(r'^(\w)\1+$', t)  # выкидываем токены из одного повторяющегося символа
#                or re.match(r'^[^a-zA-ZЁёА-я].*$', t)  # выкидываем токены начинающиеся не с буквы
#                or (t in stopwords) # выкидываем предлоги, союзы и т.п.    
#             )
#         ] 

In [7]:
import re
# from razdel import sentenize
from razdel import tokenize
from nltk.corpus import stopwords as nltk_stopwords
stopwords = set(nltk_stopwords.words('russian'))

def tokenizer(text,stopwords=stopwords):
    return [
            t.text for t in tokenize( # разбиваем текст на слова
                re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
            ) 
            if not (
               False
               or (len(t.text)<3) # выкидываем очень короткие слова
               or re.match(r'^[^a-zA-ZЁёА-я]+$', t.text) # выкидываем токены не содержащие букв
               or re.match(r'^(\w)\1+$', t.text)  # выкидываем токены из одного повторяющегося символа
               or re.match(r'^[^a-zA-ZЁёА-я].*$', t.text)  # выкидываем токены начинающиеся не с буквы
               or (t.text in stopwords) # выкидываем предлоги, союзы и т.п.    
            )
        ] 

## выполняем частотный анализ

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# tf_model = CountVectorizer(
#         min_df=.01, # выкидываем очень редкие слова
#         max_df=.25, # выкидываем очень частые слова
#         tokenizer=tokenizer, # ф-ция токенайзер
#         token_pattern=None, # отключаем дефолтный токенайзер
#         binary=True,
#     )

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_model = TfidfVectorizer(
        min_df=.01, # выкидываем очень редкие слова
        max_df=.10, # выкидываем очень частые слова
        use_idf=False, # не используем обратную частоту
        norm='l2', # нормируем TF
        tokenizer=tokenizer, # ф-ция токенайзер
        token_pattern=None, # отключаем дефолтный токенайзер
        ngram_range = (2,2)
    )

In [9]:
%%time

data_tf = tf_model.fit_transform( data['text'].str.lower() )

display(data_tf.shape)

(3196, 116)

CPU times: user 4.39 s, sys: 60.9 ms, total: 4.45 s
Wall time: 4.45 s


In [10]:
vocab = sorted(tf_model.vocabulary_)
display(len(vocab))
display(vocab)

116

['adobe flash',
 'afisha tut',
 'auto tut',
 'finance tut',
 'flash player',
 'html установлена',
 'javascript ваш',
 'realty tut',
 'sport tut',
 'wall street',
 'ближайшее время',
 'большая часть',
 'браузер поддерживает',
 'ваш браузер',
 'версия проигрывателя',
 'владимир путин',
 'внимание отключен',
 'возбуждено уголовное',
 'вторник декабря',
 'второе место',
 'глава государства',
 'главы государства',
 'говорится сообщении',
 'года назад',
 'данный момент',
 'дек риа',
 'декабря auto',
 'декабря lenta',
 'декабря sport',
 'декабря tut',
 'декабря года',
 'декабря обновлено',
 'декабря тасс',
 'дональд трамп',
 'дональда трампа',
 'друг друга',
 'избранного президента',
 'избранный президент',
 'иностранных дел',
 'конца года',
 'коренных малочисленных',
 'лет назад',
 'лиги чемпионов',
 'лошадиных сил',
 'малочисленных народов',
 'мвд россии',
 'миллиона рублей',
 'миллионов рублей',
 'млн долларов',
 'млн рублей',
 'москва дек',
 'москва декабря',
 'народов севера',
 'настояще

## кластеризируем

In [11]:
# оценки расстояний 
from sklearn.metrics.pairwise import euclidean_distances
d = euclidean_distances(data_tf)

display( d[d>0.].min(),d[d>0.].mean(),d.max(), )

0.018299992189866887

1.2168617334102432

1.4142135623730954

In [12]:
%%time

from sklearn.cluster import DBSCAN

data['cluster_id'] = DBSCAN(eps=.7,min_samples=3).fit(data_tf).labels_

display( data['cluster_id'].drop_duplicates().count() )

83

CPU times: user 74.8 ms, sys: 36.7 ms, total: 111 ms
Wall time: 111 ms


In [13]:
# номер кластера, количество объектов, метки объектов
# (cluster=-1 - некластеризованные DBSCAN объекты) 
cluster_descr = pd.concat([
        data[['cluster_id','tag']].groupby(['cluster_id'])['tag'].count(),
        data[['cluster_id','tag']].groupby(['cluster_id'])['tag'].apply(lambda s: set(s)).apply(' '.join)
    ],axis=1).reset_index()

cluster_descr.columns = ['cluster_id','count','tags']

display( cluster_descr )

,cluster_id,count,tags
0,-1,465,science culture realty incident social auto economics reclama sport health tech woman politics
1,0,921,science culture realty incident social auto economics reclama sport health tech woman politics
2,1,25,science culture auto economics sport tech politics
3,2,17,culture social auto economics tech politics
4,3,120,science culture incident social auto economics sport tech woman politics
...,...,...,...
78,77,10,incident
79,78,5,incident
80,79,3,incident
81,80,3,incident auto


In [15]:
display( data.query('cluster_id==62') )

,text,tag,cluster_id
1258,"Жителей Полоцка приглашают поучаствовать в конкурсе на лучший новогодний тюнинг\n\n29 ноября 2016 в 9:04\n\nБЕЛТА\n\nВ Полоцке пройдет открытый конкурс на лучший новогодний авторестайлинг, сообщил...",auto,62
2245,"Инцидент с ребенком произошел в автобусе №77 в Орджоникидзевском районе Перми. Водитель пропустил остановку и высадил мальчика на мороз между остановочными пунктами в лесу.\n\nПо словам очевидцев,...",incident,62
2503,"10 Декабря `16 | 16:19\n\nВ четверг, 15 декабря, на киностудии ""Ленфильм"" (Каменноостровский проспект, 10) состоится торжественное открытие III Санкт-Петербургской выставки исторической литературы...",culture,62
2586,"Вашингтон, , 09:00 — REGNUM Последнее суперлуние (совпадение полнолуния с моментом наибольшего сближения Луны и Земли) в этом году можно будет наблюдать в ночь на 14 декабря, сообщили в пресс-служ...",science,62
3076,ФедералПресс В Туве зарегистрирована ассоциация коренного малочисленного народа\nтувинцев-тоджинцев ?Тос-Чадыр? Управление министерства юстиции Республики Тува\nзарегистрировало в качестве общест...,politics,62
3093,ИА Dv-News (dv-news.com) Тувинцы-тоджинцы собрались в общественную\nорганизацию Управление министерства юстиции Республики Тува зарегистрировало\nв качестве общественной организации ассоциацию кор...,politics,62
3132,"20.07.2010. СеверИнфо (severinfo.ru) (Вологда) В Нарьян-Маре появится еще один\nпамятник ИГ ""СеверИнфо"" Архангельская обл. В Нарьян-Маре появится еще один\nпамятник Закладной камень новому памятн...",culture,62
3174,Двина Информ В Нарьян-Маре появится еще один памятник Закладной камень новому\nпамятнику оленно-транспортным батальонам будет заложен в дни празднования\n75-летия города Нарьян-Мара... Об этом со...,culture,62
